In [1]:
from imp import reload
import cProfile, time

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import cca_pytorch, cca, gp

import torch

In [8]:
T = 20
feats = 50
d = 5
K = gp.gen_kernel('squared_exp', temporal_scale=3., spatial_scale=.5)
Xt, _, _, _ = gp.embed_gp(100, feats, d, K, np.eye(feats), T, num_to_concat=1000)
CovXt = cca.calc_cross_cov_mats_from_data(Xt, 2 * T)
V_init = scipy.stats.ortho_group.rvs(feats)[:, :d]

In [12]:
t = time.time()
vp = cca_pytorch.run_cca(CovXt, d, verbose=False, init=V_init)
print(time.time() - t)
print(cca_pytorch.build_loss(CovXt, d)(vp))

13.865195274353027
tensor(-2.0888)


In [13]:
t = time.time()
va = cca.run_cca(CovXt, d, verbose=False, init=V_init)
print(time.time() - t)
print(cca.build_loss(CovXt, d)(va.ravel()))

24.356550693511963
-2.0888249702039077


In [17]:
np.linalg.eigvalsh(va.T.dot(vp))

array([-1.27734856, -1.06889401, -0.41679388,  0.50958124,  0.97753522])